### File I/O

How can we save the weights and biases (or other paramters) of our model?
How do we load those parameters for later use?

In [17]:
import torch
from torch import nn
from torch.nn import functional as F
import os
savedTensors = "data/saved_tensors"
savedParameters = "data/saved_parameters"

We can save tensors by directly invoking the `load` and `save` functions

In [10]:
x = torch.arange(4)
if not os.path.exists(savedTensors):
    os.makedirs(savedTensors)
torch.save(x, savedTensors + "/example")

In [14]:
x2 = torch.load(savedTensors + "/example")
x2

tensor([0, 1, 2, 3])

Are we able to store a list of tensors?

In [15]:
y = torch.zeros(4)
torch.save([x, y],savedTensors + "/example")
x2, y2 = torch.load(savedTensors + "/example")
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

*What about for entire models?*

We don't save entire models, but we can save model parameters.

*Why not?*

Models contain arbitrary code that cannot be naturally serialized. So it's important to generate the architechure separately.

*How can we serialize parameters?*

Simply create a normal nn.Module and use the `torch.save` function on the nn's state_dictionary()

In [16]:
# example MLP

class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.LazyLinear(256)
        self.output = nn.LazyLinear(10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

c:\Users\Scott Hamilton\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [20]:
if not os.path.exists(savedParameters):
    os.makedirs(savedParameters)
torch.save(net.state_dict(),savedParameters + '/mlp.params')

In [21]:
clone = MLP()
clone.load_state_dict(torch.load(savedParameters + '/mlp.params'))
clone.eval()

c:\Users\Scott Hamilton\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\lazy.py:178: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


MLP(
  (hidden): LazyLinear(in_features=0, out_features=256, bias=True)
  (output): LazyLinear(in_features=0, out_features=10, bias=True)
)

In [22]:
# both the original and clone values should be equal.
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])